In [11]:
# ==============================================================================
#  라이브러리 임포트
# ==============================================================================
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm # 로컬에서는 tqdm.notebook 대신 일반 tqdm 사용
from tqdm.contrib.concurrent import thread_map
import joblib

# skimage
from skimage.exposure import rescale_intensity, equalize_hist
from skimage.filters import gaussian
from skimage.restoration import denoise_bilateral
from skimage.util import img_as_ubyte
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops
from skimage.transform import resize
from scipy import ndimage

# sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.metrics import pairwise_distances, silhouette_score
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# joblib (병렬 처리를 위해)
from joblib import Parallel, delayed

# faiss (KNN 가속화를 위해)
import faiss

# 데이터 관련 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import seaborn as sns
import matplotlib.pyplot as plt

# Ensemble
from Ensemble import EnsembleFaissKNN

#DATASET_BASE_PATH = "C:/Users/bvb09/.cache/kagglehub/datasets/mikhailma/test-dataset/versions/1/Google_Recaptcha_V2_Images_Dataset"

print("✔ 모든 라이브러리 임포트 완료!")

✔ 모든 라이브러리 임포트 완료!


In [12]:
# ==============================================================================
#  데이터 로드 함수
# ==============================================================================

# 이미지 로드 함수
def load_images_from_folder(base_path):
    images = []
    labels = []
    image_folder_path = os.path.join(base_path, 'images')
    
    if not os.path.exists(image_folder_path):
        raise FileNotFoundError(f"이미지 폴더를 찾을 수 없습니다: {image_folder_path}\n"
                                f"DATASET_BASE_PATH를 올바르게 설정했는지 확인해주세요.")

    for label_name in tqdm(os.listdir(image_folder_path), desc="폴더 로드 중"):
        label_path = os.path.join(image_folder_path, label_name)
        if os.path.isdir(label_path):
            for img_name in os.listdir(label_path):
                img_path = os.path.join(label_path, img_name)
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        images.append(img)
                        labels.append(label_name)
                    else:
                        print(f"경고: {img_path} 이미지를 로드할 수 없습니다.")
                except Exception as e:
                    print(f"경고: {img_path} 로딩 중 오류 발생 - {e}")
    return pd.DataFrame({'image_data': images, 'label': labels})

def visualize_features(X_feats, y_labels, method='pca'):
    if method == 'pca':
        reducer = PCA(n_components=2)
    else:
        from sklearn.manifold import TSNE
        reducer = TSNE(n_components=2, random_state=42)

    X_reduced = reducer.fit_transform(X_feats)

    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y_labels, cmap='tab20', s=10, alpha=0.7)
    plt.title(f'Feature Distribution via {method.upper()}')
    plt.colorbar(scatter, ticks=range(len(set(y_labels))))
    plt.tight_layout()
    plt.show()

In [13]:
def edge(img_bgr):
    # 1) 강제 리사이즈
    img_bgr = cv2.resize(img_bgr, (120, 120), interpolation=cv2.INTER_AREA)

    # 2) CLAHE → Gray → Blur → Denoise → Canny
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(2.0, (8,8))
    l = clahe.apply(l)
    img_eq = cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2BGR)

    gray = cv2.cvtColor(img_eq, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3,3), 0)
    denoised = cv2.fastNlMeansDenoising(blurred, h=10, templateWindowSize=7, searchWindowSize=21)
    edges = cv2.Canny(denoised, 100, 230)

    #plt.imshow(edges, cmap='gray')
    #plt.title("Preprocessed")
    #plt.axis('off')
    #plt.show()
    return edges  # 항상 (IMG_SIZE, IMG_SIZE)

# 1) 공통: BGR → LAB → CLAHE(L) → BGR (필요 시)
def apply_clahe(img_bgr):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = cv2.createCLAHE(2.0, (8,8)).apply(l)
    return cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2BGR)

# 3) Unsharp mask (SIFT 전용)
def unsharp(img_gray):
    blurred = cv2.GaussianBlur(img_gray, (3,3), 0)
    return cv2.addWeighted(img_gray, 1.5, blurred, -0.5, 0)

# 4) Mild Gaussian blur (LBP/GLCM/Laws)
def mild_blur(img_gray):
    return cv2.GaussianBlur(img_gray, (3,3), 0)

In [14]:
def gray(img):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return image

In [15]:
def checkcol(img_bgr):
    #image = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    #image=cv2.GaussianBlur(image, ksize=(3,3), sigmaX=0)
    #image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    #image = cv2.cvtColor(img_bgr, cv2.COLOR_GRAY2BGR)
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, (8,8,8), [0,180, 0,256, 0,256])
    hist = cv2.normalize(hist, hist).flatten()
    h, s, v = cv2.split(hsv)
    rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    hist_rgb = cv2.calcHist([rgb], [0, 1, 2], None, (8,8,8), [0,256, 0,256, 0,256])
    hist_rgb = cv2.normalize(hist_rgb, hist_rgb).flatten()
    hist_bgr = cv2.calcHist([img_bgr], [0, 1, 2], None, (8,8,8), [0,256, 0,256, 0,256])
    hist_bgr = cv2.normalize(hist_bgr, hist_bgr).flatten()
    #plt.imshow(v)
    #plt.title("Color")
    #plt.axis('off')
    #plt.show()
     # ✅ 2D 시각화
    plt.figure(figsize=(10, 4))
    plt.plot(hist, color='blue')
    plt.title("Flattened HSV Histogram")
    plt.xlabel("Bin Index")
    plt.ylabel("Normalized Value")
    plt.tight_layout()
    plt.show()
    return hsv


In [16]:
tst = "C:/Users/bvb09/.cache/kagglehub/datasets/pre"
timg = load_images_from_folder(tst)
timg['processed_image_data'] = [edge(img) for img in tqdm(timg['image_data'], desc="이미지 전처리")]

이미지 전처리: 100%|██████████| 8/8 [00:00<00:00, 37.07it/s]


In [17]:
# ==============================================================================
#  피쳐 추출 함수
# ==================================================================:============

def extract_color_histogram_features(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0,1,2], None, (8,8,8), [0,180,0,256,0,256])
    return cv2.normalize(hist, hist).flatten()

def extract_glcm_features(image):
    if image is None:
        num_props = 6
        num_distances = 3 # 아래 distances 리스트 길이
        num_angles = 4    # 아래 angles 리스트 길이
        return np.zeros(num_props * num_distances * num_angles)
    
    img_glcm = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_glcm = mild_blur(img_glcm)
    img_glcm = apply_clahe(cv2.cvtColor(img_glcm, cv2.COLOR_GRAY2BGR))
    gray_image = cv2.cvtColor(img_glcm, cv2.COLOR_BGR2GRAY)

    distances = [1, 2, 3] # 예시 거리 값
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4] # 예시 각도 값
    
    try:
        glcm = graycomatrix(gray_image, distances=distances, angles=angles, symmetric=True, normed=True)
        
        props_to_extract = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
        glcm_features = []
        for prop in props_to_extract:
            glcm_features.append(graycoprops(glcm, prop).ravel())
            
        return np.concatenate(glcm_features)
    except Exception as e:
        # 오류 발생 시 처리 (예: 0으로 채워진 배열 반환)
        print(f"GLCM 추출 중 오류 발생: {e}")
        num_props = 6
        num_distances = len(distances) 
        num_angles = len(angles)    
        return np.zeros(num_props * num_distances * num_angles)

def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
     # 1) 리사이즈
    img = cv2.resize(image, (120, 120), interpolation=cv2.INTER_AREA)
    # 2) 그레이스케일 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    features = hog(gray,
                   orientations=orientations,
                   pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block,
                   block_norm='L2-Hys',
                   visualize=False,
                   transform_sqrt=True,
                   feature_vector=True)
    return features.astype(np.float32)

def extract_sift_descriptors_from_array(image):
    img = apply_clahe(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_image=unsharp(img)

    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(gray_image, None) # 키포인트와 디스크립터 계산

    return des # 디스크립터가 없을 경우 None 반환

def extract_lbp_features_from_array(image, P=8, R=1, method='uniform'):
    
    image=apply_clahe(image)
    #image=mild_blur(image)
    gray_image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # local_binary_pattern 함수는 float 타입 이미지를 선호하지만, skimage는 UBYTE도 처리
    lbp_image = local_binary_pattern(gray_image, P, R, method=method)

    # LBP 히스토그램 계산
    max_bins = P * (P - 1) + 3 if method == 'default' else P + 2 # uniform의 경우 P+2
    hist, _ = np.histogram(lbp_image.ravel(), bins=max_bins, range=(0, max_bins), density=True)
    return hist

# Laws' Texture Energy - 기존과 동일
def extract_laws_energy_features(image, window_size=15):
    image_gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray=mild_blur(image_gray)

    L5 = np.array([1, 4, 6, 4, 1], dtype=np.float32)
    E5 = np.array([-1, -2, 0, 2, 1], dtype=np.float32)
    S5 = np.array([-1, 0, 2, 0, -1], dtype=np.float32)
    W5 = np.array([-1, 2, 0, -2, 1], dtype=np.float32)
    R5 = np.array([1, -4, 6, -4, 1], dtype=np.float32)
    kernels = [L5, E5, S5, W5, R5]

    energy_features = []
    if image_gray.dtype == np.uint8:
        image_gray = image_gray.astype(np.float32)

    for k1 in kernels:
        for k2 in kernels:
            kernel = np.outer(k1, k2)
            filtered = ndimage.convolve(image_gray, kernel, mode='reflect')
            energy = np.abs(filtered)
            summed = cv2.boxFilter(energy, ddepth=-1, ksize=(window_size, window_size), normalize=False)
            energy_features.append(summed.mean())

    return np.array(energy_features, dtype=np.float32)

def learn_bovw_vocabulary(all_sift_descriptors, num_clusters=200):
    """
    Bag of Visual Words (BoVW)를 위한 시각적 단어(Vocabulary)를 학습합니다.
    """
    concatenated_descriptors = np.vstack([des for des in all_sift_descriptors if des is not None])
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    compactness, labels, centers = cv2.kmeans(concatenated_descriptors.astype(np.float32),
                                              num_clusters, None, criteria, 10, flags)
    return centers

def create_bovw_histogram(sift_descriptors, vocabulary):
    """
    SIFT 디스크립터와 학습된 Vocabulary를 사용하여 BoVW 히스토그램을 생성합니다.
    """
    num_clusters = vocabulary.shape[0]
    histogram = np.zeros(num_clusters, dtype=np.float32)

    if sift_descriptors is not None and len(sift_descriptors) > 0:
        for des in sift_descriptors:
            distances = np.linalg.norm(vocabulary - des, axis=1)
            closest_cluster_idx = np.argmin(distances)
            histogram[closest_cluster_idx] += 1
        histogram = cv2.normalize(histogram, histogram, norm_type=cv2.NORM_L2).flatten()
    return histogram

print("✔ 특징 추출 함수 정의 완료")

def extract_sift_features(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    if descriptors is None:
        descriptors = np.zeros((1, 128))
    return np.mean(descriptors, axis=0)  # 평균 풀링


✔ 특징 추출 함수 정의 완료


In [18]:
def get_features_for_split(df, image_dir, bovw_vocabulary=None, num_bovw_clusters=200):

    hog_feats = []
    color_feats = []
    lbp_feats = []
    sift_descriptors_raw = [] # SIFT 디스크립터만

    dataset_name = df.name if hasattr(df, 'name') else 'dataset'
    print(f"Loading images and extracting raw features for {dataset_name}...")

    # 이미지를 한 번 로드하고, 이를 각 특징 추출 함수에 전달
    for img_name in tqdm(df['image_name'], desc=f"Processing images for {dataset_name}"):
        img_path = os.path.join(image_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image not found at {img_path}")

        # 모든 이미지가 이미 120x120으로 리사이즈되었다고 가정하므로, 추가 리사이즈 없음
        # 다만, cv2.imread는 BGR로 읽으므로, 컬러 히스토그램을 위해 RGB 변환은 필요
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # RGB 변환은 여기서 한 번만 수행

        # 각 특징 추출 함수에 이미지 배열 전달
        hog_feats.append(extract_hog_features(image_rgb)) # HOG는 내부에서 GRAY 변환
        color_feats.append(extract_color_histogram_features(image_rgb))
        lbp_feats.append(extract_lbp_features_from_array(image_rgb)) # LBP는 내부에서 GRAY 변환
        sift_descriptors_raw.append(extract_sift_descriptors_from_array(image_rgb)) # SIFT는 내부에서 GRAY 변환


    current_bovw_vocab = bovw_vocabulary
    if current_bovw_vocab is None: # 학습 데이터에서만 Vocabulary 학습
        print(f"Learning BoVW vocabulary for {dataset_name}...")
        current_bovw_vocab = learn_bovw_vocabulary(sift_descriptors_raw, num_clusters=num_bovw_clusters)
        print(f"BoVW vocabulary learned with {current_bovw_vocab.shape[0]} clusters.")

    all_bovw_feats = []
    print(f"Creating BoVW histograms for {dataset_name}...")
    for sift_des in tqdm(sift_descriptors_raw):
        all_bovw_feats.append(create_bovw_histogram(sift_des, current_bovw_vocab))

    return (np.array(hog_feats), np.array(color_feats),
            np.array(all_bovw_feats), np.array(lbp_feats), current_bovw_vocab)


def combine_features(*feature_arrays):
    reshaped = []
    for arr in feature_arrays:
        arr = np.asarray(arr)
        if arr.ndim == 3:
            # (N, H, W) → (N, H*W)
            arr = arr.reshape(arr.shape[0], -1)
        elif arr.ndim == 1:
            # (D,) → (1, D)
            arr = arr.reshape(1, -1)
        reshaped.append(arr)
    return np.hstack(reshaped)


In [19]:
# ==============================================================================
#  분류 모델 및 학습/평가 함수
# ==============================================================================
# 유클리드 거리 기반 Faiss KNN 학습 (수정됨)
def train_faiss_knn_euclidean(X_train, y_train, n_neighbors=3):
    # 유클리드 거리 기반에서는 데이터 정규화가 필요 없습니다.
    # X_train = normalize(X_train, axis=1) # 이 줄을 주석 처리하거나 삭제합니다.
    X_train = X_train.astype(np.float32)
    D = X_train.shape[1]

    # Inner Product (IP) 대신 L2 거리를 사용하는 인덱스를 생성합니다.
    index = faiss.IndexFlatL2(D) # <-- 변경된 부분
    index.add(X_train)

    return index, y_train, n_neighbors

# 유클리드 거리 기반 Faiss KNN 예측 (수정됨)
def predict_faiss_knn_euclidean(index, y_train_labels, n_neighbors, X_test):
    # 유클리드 거리 기반에서는 테스트 데이터 정규화도 필요 없습니다.
    # X_test = normalize(X_test, axis=1) # 이 줄을 주석 처리하거나 삭제합니다.
    X_test = X_test.astype(np.float32)

    # search 결과는 D (거리)와 I (인덱스)입니다.
    # L2 거리이므로 거리가 작을수록 가까운 것입니다.
    distances, indices = index.search(X_test, n_neighbors) # <-- 이름 변경 (similarities -> distances)

    y_pred = []
    for i in range(len(X_test)):
        neighbor_labels = y_train_labels[indices[i]]
        # 거리 기반이므로 가장 가까운 이웃들의 레이블을 통해 다수결 투표
        unique_labels, counts = np.unique(neighbor_labels, return_counts=True)
        predicted_label = unique_labels[np.argmax(counts)]
        y_pred.append(predicted_label)
    return np.array(y_pred)

# test() 함수
def test(model_tuple, X_test, y_test, average='weighted'):
    faiss_index, y_train_labels, n_neighbors = model_tuple
    
    print(f"  ▶ KNN 예측 중 (Faiss 사용, k={n_neighbors})...")
    y_pred = predict_faiss_knn_euclidean(faiss_index, y_train_labels, n_neighbors, X_test)
    print("  ✔ 예측 완료.")
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average=average, zero_division=0)
    rec  = recall_score(y_test, y_pred, average=average, zero_division=0)
    print(f"[Test Accuracy] {acc:.4f}")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    return y_pred
print("✔ 분류 모델 및 학습/평가 함수 정의 완료 (Faiss 적용).")

✔ 분류 모델 및 학습/평가 함수 정의 완료 (Faiss 적용).


In [20]:
DATASET_BASE_PATH = "C:/Users/bvb09/recaptcha-dataset"

try:
    images_df = load_images_from_folder(DATASET_BASE_PATH)
    print(f"✔ 총 {len(images_df)}개의 이미지 로드 완료.")
except FileNotFoundError as e:
    print(f"오류: {e}")

# 2. 라벨 인코딩
le = LabelEncoder()
images_df['label_encoded'] = le.fit_transform(images_df['label'])
print(f"✔ 라벨 인코딩 완료. 클래스: {le.classes_}")

# 전체 이미지 데이터에 대해 전처리 수행
#print("⏳ 전체 이미지 전처리 중...")
#images_df['processed_image_data'] = [preprocess_image(img) for img in tqdm(images_df['image_data'], desc="이미지 전처리")]
#print("✔ 전체 이미지 전처리 완료.")


폴더 로드 중: 100%|██████████| 10/10 [00:05<00:00,  1.69it/s]

✔ 총 10292개의 이미지 로드 완료.
✔ 라벨 인코딩 완료. 클래스: ['Bicycle' 'Bridge' 'Bus' 'Car' 'Chimney' 'Crosswalk' 'Hydrant'
 'Motorcycle' 'Palm' 'Traffic Light']


In [21]:
# 전체 이미지 데이터에 대해 기본 특징 미리 추출
print("⏳ 기본 특징 추출 중 (Color, HOG, LBP, SIFT descriptors)...")
features_color_all = np.array([extract_color_histogram_features(img) for img in tqdm(images_df['image_data'], desc="Color Histogram 추출")])

⏳ 기본 특징 추출 중 (Color, HOG, LBP, SIFT descriptors)...


Color Histogram 추출: 100%|██████████| 10292/10292 [00:00<00:00, 18939.57it/s]


In [22]:
features_lbp_all = np.array([extract_lbp_features_from_array(img) for img in tqdm(images_df['image_data'], desc="LBP 추출")])


LBP 추출: 100%|██████████| 10292/10292 [00:16<00:00, 612.75it/s]


In [23]:
hog_list = [
    extract_hog_features(img)
    for img in tqdm(images_df['image_data'], desc="HOG 추출")
]
features_hog_all = np.vstack(hog_list)  # shape = (n_images, hog_dim)
print(features_hog_all.shape)


HOG 추출: 100%|██████████| 10292/10292 [00:29<00:00, 348.50it/s]


(10292, 7056)


In [ ]:
#features_glcm_all = np.array([extract_glcm_features(img) for img in tqdm(images_df['image_data'], desc="GLCM 추출")])


GLCM 추출:   8%|▊         | 841/10292 [00:52<09:47, 16.07it/s]


KeyboardInterrupt: 

In [ ]:
features_laws_all = np.array([extract_laws_energy_features(img) for img in tqdm(images_df['image_data'], desc="Laws' Texture 추출")])


Laws' Texture 추출: 100%|██████████| 10168/10168 [00:54<00:00, 185.79it/s]


In [ ]:
features_sift_all = [extract_sift_features(img) for img in tqdm(images_df['image_data'], desc="SIFT 추출")]

SIFT Descriptors 추출: 100%|██████████| 10168/10168 [00:42<00:00, 237.53it/s]


In [ ]:
features_edge_all = np.array([edge(img) for img in tqdm(images_df['image_data'], desc="canny 추출")])

canny 추출: 100%|██████████| 10168/10168 [03:53<00:00, 43.54it/s]


In [ ]:
# 사용할 특징 조합 정의
#feature_names = ['color', 'hog', 'lbp', 'sift']
all_feature_combinations_tuples = []
#from itertools import combinations
#for i in range(1, len(feature_names) + 1):
#    for combo in combinations(feature_names, i):
#        all_feature_combinations_tuples.append(combo)

# 사용자가 요청한 특정 조합 추가 (필요시)
user_requested_combinations = [
 ('color'), ('color', 'sift'), ('color', 'lbp'), ('sift','lbp'), ('color', 'lbp', 'hog'), ('color', 'lbp', 'sift'),
 ('color', 'law', 'hog'), ('color', 'law', 'sift'), ('color', 'glcm', 'hog'),
 ('color', 'lbp', 'sift', 'hog')
]
all_feature_combinations_tuples.extend(user_requested_combinations)
# # 중복 제거
# all_feature_combinations_tuples = sorted(list(set(all_feature_combinations_tuples)))


print(f"\n▶ 총 {len(all_feature_combinations_tuples)}개의 특징 조합에 대해 교차 검증을 수행합니다.")
for combo in all_feature_combinations_tuples:
    print(f"  - {combo}")

# 결과를 저장할 딕셔너리
results = {}

X_indices = np.arange(len(images_df)) # StratifiedKFold에 사용할 인덱스
y_labels = images_df['label_encoded'].values

# Stratified K-Fold 준비
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 노트북의 기본 5-fold 사용
print("\n✔ Stratified K-Fold 설정 완료 (5-Fold)")

# 교차 검증 루프
num_bovw_clusters = 200 # BoVW 클러스터 수 (노트북의 get_features_for_split 함수 내 num_bovw_clusters=200 참고, 줄여서 테스트)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_indices, y_labels)):
    print(f"\n=============== FOLD {fold+1}/5 ================")
    y_train_fold, y_val_fold = y_labels[train_idx], y_labels[val_idx]

    # 현재 폴드의 SIFT 디스크립터 (훈련 데이터용, 검증 데이터용)
    #sift_descriptors_train_fold = [sift_descriptors_list_all[i] for i in train_idx]
    #sift_descriptors_val_fold = [sift_descriptors_list_all[i] for i in val_idx]

    #print(f"  ⏳ BoVW Vocabulary 학습 중 (Fold {fold+1})")
    # SIFT 디스크립터가 하나도 없는 경우를 방지
    #valid_sift_descriptors_train_fold = [d for d in sift_descriptors_train_fold if d is not None and len(d) > 0]
    #bovw_vocabulary = learn_bovw_vocabulary(valid_sift_descriptors_train_fold, num_clusters=num_bovw_clusters)


    # 각 특징 조합에 대해 학습 및 평가
    for feature_combo_tuple in all_feature_combinations_tuples:
        feature_combo_name = '+'.join(feature_combo_tuple)
        print(f"\n--- 조합: {feature_combo_name} (Fold {fold+1}) ---")
        
        X_train_features_list = []
        X_val_features_list = []
        
        if 'color' in feature_combo_tuple:
            X_train_features_list.append(features_color_all[train_idx])
            X_val_features_list.append(features_color_all[val_idx])
        #if 'edge' in feature_combo_tuple:
         #   X_train_features_list.append(features_edge_all[train_idx])
          #  X_val_features_list.append(features_edge_all[val_idx])
        if 'hog' in feature_combo_tuple:
            X_train_features_list.append(features_hog_all[train_idx])
            X_val_features_list.append(features_hog_all[val_idx])
        if 'lbp' in feature_combo_tuple:
            X_train_features_list.append(features_lbp_all[train_idx])
            X_val_features_list.append(features_lbp_all[val_idx])
        #if 'glcm' in feature_combo_tuple:
         #   X_train_features_list.append(features_glcm_all[train_idx])
          #  X_val_features_list.append(features_glcm_all[val_idx])
        if 'laws' in feature_combo_tuple:
            X_train_features_list.append(features_laws_all[train_idx])
            X_val_features_list.append(features_laws_all[val_idx])    
        if 'sift' in feature_combo_tuple:
            X_train_features_list.append(features_sift_all[train_idx])
            X_val_features_list.append(features_sift_all[val_idx])

        if not X_train_features_list: # 특징이 하나도 선택되지 않은 경우 (일어날 일 없지만 방어 코드)
            print(f"  ⚠️ 경고: 특징이 선택되지 않았습니다 ({feature_combo_name}). 건너뜁니다.")
            continue

        # 특징 결합
        X_train_combined = combine_features(*X_train_features_list)
        X_val_combined = combine_features(*X_val_features_list)
        
        # 데이터 정규화 (필요시 추가 - 예: StandardScaler 또는 MinMaxScaler)
        # scaler = StandardScaler()
        # X_train_combined = scaler.fit_transform(X_train_combined)
        # X_val_combined = scaler.transform(X_val_combined)
        # print("  ℹ️ 특징 정규화 적용됨.")

        print(f"  ⏳ Faiss KNN 모델 학습 중 (k=3, Fold {fold+1}, 조합 {feature_combo_name})...")
        # NaN 또는 Inf 값 확인 및 처리 (중요!)
        if np.isnan(X_train_combined).any() or np.isinf(X_train_combined).any():
            print(f"  ⚠️ 경고: X_train_combined에 NaN 또는 Inf 값이 포함되어 있습니다. 0으로 대체합니다. (조합: {feature_combo_name}, Fold: {fold+1})")
            X_train_combined = np.nan_to_num(X_train_combined, nan=0.0, posinf=0.0, neginf=0.0) # 또는 다른 대체 전략
        if np.isnan(X_val_combined).any() or np.isinf(X_val_combined).any():
            print(f"  ⚠️ 경고: X_val_combined에 NaN 또는 Inf 값이 포함되어 있습니다. 0으로 대체합니다. (조합: {feature_combo_name}, Fold: {fold+1})")
            X_val_combined = np.nan_to_num(X_val_combined, nan=0.0, posinf=0.0, neginf=0.0)

        # 특징 벡터가 비어있거나 모든 값이 0인 경우를 처리 (L2 정규화 등에서 문제 발생 가능)
        if X_train_combined.shape[1] == 0:
            print(f"  ⚠️ 경고: 훈련 특징 벡터가 비어있습니다. (조합: {feature_combo_name}, Fold: {fold+1}). 건너뜁니다.")
            continue
        
        # Faiss는 float32 타입을 요구함
        X_train_combined = X_train_combined.astype(np.float32)
        X_val_combined = X_val_combined.astype(np.float32)

        try:
            faiss_index, train_labels_for_pred, n_neighbors_actual = train_faiss_knn_euclidean(X_train_combined, y_train_fold, n_neighbors=3)
            
            print(f"  ▶ KNN 예측 중 (Faiss 사용, k={n_neighbors_actual}, Fold {fold+1}, 조합 {feature_combo_name})...")
            y_pred_fold = predict_faiss_knn_euclidean(faiss_index, train_labels_for_pred, n_neighbors_actual, X_val_combined)
            
            acc = accuracy_score(y_val_fold, y_pred_fold)
            fsc = f1_score(y_val_fold,y_pred_fold, average= 'weighted')
            print(f"  ✔ 예측 완료. Accuracy: {acc:.4f} (Fold {fold+1}, 조합 {feature_combo_name})")
            print(f"  ✔ 예측 완료. F1-Score: {fsc:.4f} (Fold {fold+1}, 조합 {feature_combo_name})")
            
            if feature_combo_name not in results:
                results[feature_combo_name] = []
            results[feature_combo_name].append(acc)
        except Exception as e:
            print(f"  ❌ 오류 발생 (모델 학습/예측 중, Fold {fold+1}, 조합 {feature_combo_name}): {e}")
            if feature_combo_name not in results: # 오류 발생 시에도 키는 만들어 둠
                results[feature_combo_name] = []
            results[feature_combo_name].append(np.nan) # 오류 발생 시 NaN으로 기록


# 최종 결과 집계 및 출력
print("\n\n=============== 최종 교차 검증 결과 ================")
for combo_name, acc_list in results.items():
    valid_acc_list = [acc for acc in acc_list if not np.isnan(acc)]
    if valid_acc_list:
        mean_acc = np.mean(valid_acc_list)
        std_acc = np.std(valid_acc_list)
        print(f"특징 조합: {combo_name}")
        for i, acc_fold in enumerate(acc_list):
             print(f"  Fold {i+1} Accuracy: {acc_fold:.4f}" if not np.isnan(acc_fold) else f"  Fold {i+1} Accuracy: Error")
        print(f"  >> 평균 정확도: {mean_acc:.4f} (표준편차: {std_acc:.4f})")
    else:
        print(f"특징 조합: {combo_name} - 모든 폴드에서 오류 발생 또는 유효한 결과 없음.")
    print("-" * 30)

# 가장 성능이 좋은 조합 찾기 (평균 정확도 기준)
if results:
    sorted_results = sorted(results.items(), key=lambda item: np.nanmean(item[1]) if item[1] else -1, reverse=True)
    print("\n🏆 최고 성능 조합 (평균 정확도 기준):")
    if sorted_results and np.nanmean(sorted_results[0][1]):
         best_combo_name, best_acc_list = sorted_results[0]
         print(f"  {best_combo_name}: 평균 정확도 = {np.nanmean(best_acc_list):.4f}")
    else:
        print("  유효한 결과를 가진 조합이 없습니다.")


▶ 총 15개의 특징 조합에 대해 교차 검증을 수행합니다.
  - color
  - edge
  - ('color', 'edge')
  - ('color', 'sift')
  - ('color', 'lbp')
  - ('color', 'lbp', 'hog')
  - ('color', 'lbp', 'sift')
  - ('color', 'edge', 'lbp')
  - ('color', 'law', 'hog')
  - ('color', 'law', 'sift')
  - ('color', 'glcm', 'hog')
  - ('color', 'glcm', 'sift')
  - ('color', 'edge', 'sift', 'lbp')
  - ('color', 'lbp', 'sift', 'hog')
  - ('color', 'glcm', 'sift', 'hog', 'law')

✔ Stratified K-Fold 설정 완료 (5-Fold)

=============== FOLD 1/5 ================
  ⏳ BoVW Vocabulary 학습 중 (Fold 1)

--- 조합: c+o+l+o+r (Fold 1) ---
  ⏳ Faiss KNN 모델 학습 중 (k=3, Fold 1, 조합 c+o+l+o+r)...
  ▶ KNN 예측 중 (Faiss 사용, k=3, Fold 1, 조합 c+o+l+o+r)...
  ✔ 예측 완료. Accuracy: 0.3682 (Fold 1, 조합 c+o+l+o+r)
  ✔ 예측 완료. F1-Score: 0.3678 (Fold 1, 조합 c+o+l+o+r)

--- 조합: e+d+g+e (Fold 1) ---
  ⏳ Faiss KNN 모델 학습 중 (k=3, Fold 1, 조합 e+d+g+e)...
  ▶ KNN 예측 중 (Faiss 사용, k=3, Fold 1, 조합 e+d+g+e)...
  ✔ 예측 완료. Accuracy: 0.1111 (Fold 1, 조합 e+d+g+e)
  ✔ 예측 완료. F1-Score: 0.0764